In [1]:
import pickle
from qiskit import BasicAer, execute
from qiskit.quantum_info.states.measures import state_fidelity

def reverseBits(num,bitSize): 
    binary = bin(num) 
    reverse = binary[-1:1:-1] 
    reverse = reverse + (bitSize - len(reverse))*'0'
    return int(reverse,2)

def simulate_circ(circ, simulator='statevector_simulator'):
    backend = BasicAer.get_backend(simulator)
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputstate_ordered = [0 for sv in outputstate]
    for i, sv in enumerate(outputstate):
        reverse_i = reverseBits(i,len(circ.qubits))
        outputstate_ordered[reverse_i] = sv
    return outputstate_ordered

dirname = './data'
full_circ = pickle.load(open( '%s/full_circ.p'%dirname, 'rb' ))
original_sv = simulate_circ(full_circ)
reconstructed_sv = pickle.load(open( '%s/reconstructed.p'%dirname, 'rb' ))

print('fid = ',state_fidelity(original_sv,reconstructed_sv))

fid =  0.9999999999999964


In [11]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute
import numpy as np

def simulate_circ(circ, simulator):
    backend = BasicAer.get_backend('statevector_simulator')
    job = execute(circ, backend=backend)
    result = job.result()
    outputstate = result.get_statevector(circ)
    outputstate_ordered = [0 for sv in outputstate]
    for i, sv in enumerate(outputstate):
        reverse_i = reverseBits(i,len(circ.qubits))
        outputstate_ordered[reverse_i] = sv
    if simulator == 'sv':
        return outputstate_ordered
    else:
        output_prob = [np.power(np.absolute(x),2) for x in outputstate_ordered]
        return output_prob

q = QuantumRegister(4,'q')
circ = QuantumCircuit(q)
circ.x(0)
circ.x(2)

print(circ)

sv = simulate_circ(circ,'prob')
for i,sv in enumerate(sv):
    bin_i = bin(i)[2:].zfill(4)
    print(bin_i,':',sv)

        ┌───┐
q_0: |0>┤ X ├
        └───┘
q_1: |0>─────
        ┌───┐
q_2: |0>┤ X ├
        └───┘
q_3: |0>─────
             
0000 : 0.0
0001 : 0.0
0010 : 0.0
0011 : 0.0
0100 : 0.0
0101 : 0.0
0110 : 0.0
0111 : 0.0
1000 : 0.0
1001 : 0.0
1010 : 1.0
1011 : 0.0
1100 : 0.0
1101 : 0.0
1110 : 0.0
1111 : 0.0


In [2]:
import numpy as np
dict_0 = {'00':0.2,'01':0.3,'10':0.4,'11':0.5}
dict_1 = {'00':0.2,'01':0.3,'10':0.4,'11':0.5}
np.kron(dict_0,dict_1)

TypeError: unsupported operand type(s) for *: 'dict' and 'dict'